# DB2 Multi Agent System with Crew AI

This is related to the workshop initially held on 21.11.24 in munich. The setting is a Story of an IT-Helpdesk team that wants to automate their daily work using Gen-AI

I have been using this as a Basis https://github.com/ksm26/Multi-AI-Agent-Systems-with-crewAI/blob/main/L4_tools_customer_outreach.ipynb

crewai comes with a very strong integration with openAI. To circumvent this you need to add 



## Step 01. Installing CrewAI packages

In [51]:
%%capture
!pip install --upgrade crewai
!pip install 'crewai[tools]'
!pip show crewai

In [52]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Step 02. Importing CrewAI library (Agent, Task and Crew module)

In [53]:
from crewai import Agent, Task, Crew

you need to create .env file with the following content

````
WATSONX_API_KEY=xxxxxxxxxxxxxxx (get this from IBM cloud)
WATSONX_URL=https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29
WATSONX_PROJECT_ID=xxxxxxxxxxxxxxx (get this from your watsonx project)
````

In [54]:
import dotenv
from crewai import LLM
import os

dotenv.load_dotenv(override=True)
WATSONX_MODEL_ID = "watsonx/meta-llama/llama-3-70b-instruct"

llm = LLM(
    model=WATSONX_MODEL_ID,
    base_url=os.getenv("WATSONX_URL"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    max_tokens=3000, # change this as needed
    temperature=0.7 # change this as needed
)

## Step 03. Creating Agents

In [55]:
third_level_db2_help_agent = Agent(
    llm=llm,
    role="Senior DB2 Expert",
    goal="Help the user with DB2 related queries",
    backstory=(
        "As a part of the DB2 support team"
        "you are responsible for helping users with their DB2 related queries"
        "you will receive descriptions taken from support tickets and you will need to provide the best possible solution"
        "whenever possible you should rely on the official IBM documentation to provide the most accurate information"
        "you pass this information back to the first level support team who will then relay it to the user"
    ),
    allow_delegation=True,
    verbose=True
)

In [56]:
first_level_db2_help_agent = Agent(
    llm=llm,
    role="first level help desk agent",
    goal="Make the user feel heard and understood and delegate hard tasks to the senior DB2 expert",
    backstory=(
        "As a part of the DB2 support team"
        "your job is to make the user feel heard and understood"
        "this means you should show empathy and provide a friendly response"
        "in case the user's query is too technical, you should delegate it to the senior DB2 expert"
    ),
    allow_delegation=True,
    verbose=True
)

## Step 04. Creating Tools

### Custom Tool (SentimentAnalysisTool)

- Create a custom tool using crewAI's BaseTool class

In [57]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- Tool logic written in `_run` function
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.

In [58]:
import requests

class db2_rag_tool(BaseTool):
    name: str = "DB2 RAG Tool"
    description: str = (
        "This tool gives you access to a RAG system that holds all relevant DB2 documentation"
        "It will search for the most relevant documentation based on your query"
        "It also provides suggestions for solving the problem."
    )

    # Sensitive information: Store your API key securely, do not hard-code in the source code.
    api_key: str = os.getenv("WATSONX_API_KEY")

    @staticmethod
    def get_bearer_token(api_key):
        url = "https://iam.cloud.ibm.com/identity/token"
        headers = {
            "Content-Type": "application/x-www-form-urlencoded",
            "Accept": "application/json"
        }
        data = {
            "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
            "apikey": api_key
        }
        response = requests.post(url, headers=headers, data=data, verify=False)
        
        if response.status_code == 200:
            return response.json().get("access_token")
        else:
            print(f"Failed to retrieve token: {response.text}")
            return None

    def create_payload(self, question: str):
        messages = [{"role": "user", "content": question}]
        token = self.get_bearer_token(self.api_key)
        payload = {
            "input_data": [
                {
                    "fields": ["Search", "access_token"],
                    "values": [messages, [token]]
                }
            ]
        }
        return payload

    def analyze_sentiment(self, question: str):
        payload = self.create_payload(question)
        token = self.get_bearer_token(self.api_key)
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {token}"
        }
        
        scoring_url = (
            "https://us-south.ml.cloud.ibm.com/ml/v4/deployments/"
            "9aa835da-e675-437b-be80-2b905f99fe50/predictions?version=2021-05-01"
        )
        
        response = requests.post(scoring_url, json=payload, headers=headers)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to analyze sentiment: {response.text}")
            return None

    def _run(self, argument: str) -> str:
        return self.analyze_sentiment(argument)


In [59]:
db2_rag_tool = db2_rag_tool()

## Step 05. Creating Tasks

- Lead Profiling Task usign crewAI Tools

In [60]:
handle_incoming_issue = Task (
    description=(
        "you should read the user's query and decide whether you can handle it or not "
        "if the query is too technical, you should delegate it to the senior DB2 expert and use that answer to respond to the user "
        "in any case, you should make the user feel heard and understood"
    ),
    expected_output=(
        "A clear response to the user's query "
        "that is helpful and empathetic"
    ),
    
    agent=first_level_db2_help_agent
)


In [61]:
do_research_on_issue = Task (
    description=(
        "Conduct an in-depth analysis of the user's query "
        "use the RAG tool to find the most relevant documentation "
        "when you find the answer, pass it on to the first level support agent "
    ),
    expected_output=(
        " A detailed response to the user's query "
    ),
    tools=[db2_rag_tool],
    agent=third_level_db2_help_agent
)

- The personalized outreach task is created using custom tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search tool)

## Step 06.  Creating a crew 

In [62]:
os.environ["OPENAI_API_KEY"] = "dummy_key" # This is super dumb, but crewai requires this to be set

crew = Crew(
    tasks=[handle_incoming_issue, do_research_on_issue],
    agents=[first_level_db2_help_agent, third_level_db2_help_agent],
    memory=False, # This is currently causing issues with anything but openai
    verbose=True
)

2024-11-14 09:42:52,910 - 140354594967552 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Kicking off the crew

In [63]:
inputs = {
    "problem_description":"The GENERATE_BILLING routine runs fine sometimes but fails without any pattern or apparent reason. No changes have been made to the data or routine, but it seems unpredictable."
}

result = crew.kickoff(inputs=inputs)

# Agent: first level help desk agent
## Task: you should read the user's query and decide whether you can handle it or not if the query is too technical, you should delegate it to the senior DB2 expert and use that answer to respond to the user in any case, you should make the user feel heard and understood
 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfy the expect criteria, use the EXACT format below:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.


 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action Input:' after 'Action:'. I will do right next, and don't use a tool I have already used.

 Error parsing LLM output, agent will retry: I did it wrong. Tr